### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 27 апреля 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 27 апреля, -4 балла после 08:30 4 мая, -6 баллов после 08:30 11 мая, -8 баллов после 08:30 18 мая.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [77]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [78]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2) / (l_s * (l_s + r_s)) + (
            r_c ** 2) / (r_s * (l_s + r_s)), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c/l_s*np.log2(l_c/l_s), axis=1)-np.sum(r_c/r_s*np.log2(r_c/r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(
            r_c / (l_s + r_s), axis=1)
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #Отсортировали x и y с помощью нашей функции 
        x_sort, y_sort=self.__sort_samples(x,y)
        #записываем количество классов в class_num
        class_num=self.num_class
        
        #отсекаем справа и слева куски, по которым точно не пройдет граница.
        #ищем индексы, в которых y меняется, и добавляем смещение cut,
        # чтобы индексы были как в y_sort, записали это в r_border_ids
        cut = np.int(self.min_samples_split / 2 - 1)
        y_sort_slit = y_sort[cut:-cut] if cut != 0 else y_sort
        r_border_ids = np.where(y_sort_slit[:-1] != y_sort_slit[1:])[0] + (cut + 1)
        
        # если же класс не сменяется вообще, то выходим из функции
        if len(r_border_ids) == 0:
            return np.inf, None
        
        # Формируем массив one_hot_code - каждая строка отвечает
        # за границу, по которой потенциально может пройти разбиение.
        # Столбцы соответствуют классам. В столбце поднята единица,
        # если разбиение прошло по данному классу.
        eq_el_count = r_border_ids - np.append([cut], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_num))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     y_sort[r_border_ids - 1]] = 1
        
        # В class_increments получаем сколько элементов было в группах
        # включая первые min_samples_split элементов.
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(y_sort[:cut], minlength=class_num)
        
        
        #находим количество классов слева и справа в разбиении 
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y_sort, minlength=class_num) - l_class_count
    
        # а также количество объектов слева и справа в разбиении
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = y_sort.shape[0] - l_sizes
        
        # подсчитываем меру неопределенности и находим под каким индексом лежит минимальное значение этой меры
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        indx = np.argmin(gs)
        
        # Получаем индекс left_el_id, после которого проходит разбиение.
        left_el_id = l_sizes[indx][0]
        
        # Возвращаем неопределенность, и пороговое значение.
        return gs[indx], (x_sort[left_el_id - 1] + x_sort[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        #создаем листовую вершину, если не достигли конца.
        if (self.max_depth is not None and self.max_depth<=depth) or \
            y.size < self.min_samples_split or \
            self.sufficient_share <= np.bincount(y).argmax() / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
        #находим группs в выборке
        threshold = np.array([self.__find_threshold(x[:, i], y) for i in self.get_feature_ids(x.shape[1])])
        
        #находим группу с минимальныыми значениями.
        best_id = threshold[:, 0].argmin()
        best_threshold = threshold[best_id, 1]
        #если узел создался и нашлась группа, то спускаемся по дереву дальше, если нет, то создаем последний лист.
        if best_threshold == None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
        else:
            x_l, x_r, y_l, y_r =self.__div_samples(x, y, best_id, best_threshold)
            if x_l.size == 0 or x_r.size == 0:
                self.tree[node_id] = (self.LEAF_TYPE,np.bincount(y).argmax(),np.bincount(y).astype(float) / y.size)
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE,best_id, best_threshold)
                self.__fit_node(x_l, y_l,2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
            if self.G_function == self.__gini:
                g=(1 - np.sum(np.unique(y, return_counts=True)[1]** 2 / y.size ** 2))
                g_l=(1 - np.sum(np.unique(y_l, return_counts=True)[1] ** 2 / y_l.size ** 2))
                g_r=(1 - np.sum(np.unique(y_r, return_counts=True)[1] ** 2 / y_r.size ** 2))
                self.feature_importances_[best_id] += y.size*g - y_l.size*g_l - y_r.size*g_r
            if self.G_function == self.__entropy:
                g=-(np.sum((np.unique(y, return_counts=True)[1] / y.size) * np.log2(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=-(np.sum((np.unique(y_l, return_counts=True)[1] / y_l.size) * np.log2(np.unique(y_l, return_counts=True)[1] / y_l.size)))
                g_r=-(np.sum((np.unique(y_r, return_counts=True)[1] / y_r.size) * np.log2(np.unique(y_r, return_counts=True)[1] / y_r.size)))
                self.feature_importances_[best_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)   
            elif self.G_function == self.__misclass:
                g=(1 - np.max(np.sum(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=(1 - np.max(np.sum(np.unique(y_l, return_counts=True)[1] / y.size)))
                g_r=(1 - np.max(np.sum(np.unique(y_r, return_counts=True)[1] / y.size)))
                self.feature_importances_[best_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)
                
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [79]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [80]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [81]:
my_clf.fit(X_train,y_train)
clf.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [82]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

1.0
1.0


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [83]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.85 ms


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [84]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 14.2 ms


In [85]:
my_clf.feature_importances_

array([0.02534774, 0.        , 0.01221591, 0.        , 0.        ,
       0.        , 0.26623152, 0.        , 0.        , 0.24873415,
       0.        , 0.        , 0.10551755])

## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [86]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
for i in df.columns:
    if type(df[i][0])==np.float64 or type(df[i][0])==np.int64 or type(df[i][0])==float:
        continue
    print(type(df[i][0]))
    df.drop(i,axis=1, inplace=True)
df.info()

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 190 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(3)
memory usage: 12.1+ MB


In [87]:
for i in df.columns:
    if df[i].isnull().sum()>500:
        df.drop(i,axis=1, inplace=True)

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

In [88]:
df=df.fillna(-9999)
df.drop('id', axis=1)
y = df.loc[:, 'gender'].values
X=df.drop('gender', axis=1).iloc[:,0:].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [90]:
from sklearn.metrics import f1_score
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)
print(f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro'))
print(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro'))

0.9789030745093146
0.9789056396170333


## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

